In [ ]:
import os
import pandas as pd
from google.colab import auth
from datetime import datetime
import requests
import json
auth.authenticate_user()
!gsutil -m cp -r gs://predict_cfs/revised_metadata .
!mkdir end_data
def get_elevation_Data(l1, l2, key):
  url = "https://maps.googleapis.com/maps/api/elevation/json?locations={},{}&key={}".format(l1, l2, key)
  resp = requests.get(url)
  return json.loads(resp.text)
def get_soil_grids(l1, l2):
  url = "https://rest.soilgrids.org/soilgrids/v2.0/properties/query?lon={}&lat={}&property=bdod&property=cec&property=cfvo&property=clay&property=nitrogen&property=ocd&property=ocs&property=phh2o&property=sand&property=silt&property=soc&depth=0-5cm&depth=0-30cm&depth=5-15cm&depth=15-30cm&depth=30-60cm&depth=60-100cm&depth=100-200cm&value=Q0.05&value=Q0.5&value=Q0.95&value=mean&value=uncertainty"
  final_url = url.format(l2, l1)
  resp = requests.get(final_url)
  return json.loads(resp.text)



In [ ]:
!mkdir end_dat

In [ ]:
def loop_through(key, dir="revised_data", start_idx=246, end_idx=1000):
  files = os.listdir(dir) 
  for f in range(start_idx, end_idx):
    with open(os.path.join(dir, files[f])) as file1:
      data = json.load(file1)
    if "latitude" in data:
      lat = data["latitude"]
      lng = data["longitude"]
      data["goog_el"] = get_elevation_Data(lat, lng, key)
      data["soil"] = get_soil_grids(lat, lng)
      with open(os.path.join("end_dat", files[f]), "w+") as y:
        json.dump(data, y)
    else: 
      print("Error for" + files[f])
    
!mkdir end_dat
key = "put_key_here"
loop_through(key, "revised_metadata")

In [ ]:
!gsutil cp -r end_dat gs://predict_cfs/meta_soil

In [ ]:
!gsutil cp -r end_dat gs://predict_cfs/meta_soil

'{"detail":[{"loc":["query","lat"],"msg":"ensure this value is greater than or equal to -90.0","type":"value_error.number.not_ge","ctx":{"limit_value":-90.0}}]}'

In [ ]:
with open('revised_metadata/1111212stations.json') as y:
  data =json.load(y)
print(data)


{'gage_id': 1111212, 'stations': [{'station_id': 'SFZ', 'dist': 18.140337131301568, 'cat': 'ASOS', 'missing_temp': 212135, 'missing_precip': 192893}, {'station_id': 'KSFZ', 'dist': 18.17133860668529, 'cat': 'ASOS', 'missing_temp': 212135, 'missing_precip': 192893}, {'station_id': 'ORH', 'dist': 31.860095867215463, 'cat': 'ASOS', 'missing_temp': 225224, 'missing_precip': 209662}, {'station_id': 'KORH', 'dist': 31.98685035204759, 'cat': 'ASOS', 'missing_temp': 225224, 'missing_precip': 209662}, {'station_id': 'KOWD', 'dist': 39.53058702564567, 'cat': 'ASOS', 'missing_temp': 232873, 'missing_precip': 219306}, {'station_id': 'OWD', 'dist': 39.574177951536804, 'cat': 'ASOS', 'missing_temp': 232873, 'missing_precip': 219306}, {'station_id': 'KPVD', 'dist': 40.02847126174439, 'cat': 'ASOS', 'missing_temp': 233270, 'missing_precip': 218330}, {'station_id': 'PVD', 'dist': 40.02847126174439, 'cat': 'ASOS', 'missing_temp': 233270, 'missing_precip': 218330}, {'station_id': 'MQE', 'dist': 44.958089

## Getting data

In [ ]:
!gsutil du gs://predict_cfs/revised_metadata_final | wc -l

3882


In [ ]:
!gsutil du gs://predict_cfs/meta_data/distance3| wc -l

3882
